In [4]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, roc_curve, auc, precision_score, recall_score, f1_score
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# pip install spacy-universal-sentence-encoder
import spacy_universal_sentence_encoder
nlp = spacy_universal_sentence_encoder.load_model('en_use_lg')

from functools import reduce
from tqdm import tqdm
import gensim
import gensim.corpora as corpora

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from Preprocessing import Preprocessor
preprocessor = Preprocessor(0)
tqdm.pandas()

# import necessary libraries
import tensorflow_hub as hub
  
# Load pre-trained universal sentence encoder model
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")



In [5]:
ted = pd.read_csv("ted_joined.csv")
ted.head(5)

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,...,speaker_occupation,tags,title,urlurl,views,transcript,urlurl.1,transcript_cleaned,tokens,transcript_n_entries
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,...,Author/educator,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,"good morning. you? it great, it? blown away wh...","['good', 'morning.', 'you?', 'it', 'great,', '...",1484
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,...,Climate advocate,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,"thank much, chris. truly great honor opportuni...","['thank', 'much,', 'chris.', 'truly', 'great',...",1012
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,...,Technology columnist,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,"(music: the sound silence, simon & garfunkel)h...","['(music:', 'the', 'sound', 'silence,', 'simon...",1673
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,...,Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,todayhappyheard sustainable development save u...,"['todayhappyheard', 'sustainable', 'developmen...",1624
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,...,Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,"10 years ago, took task teach global developme...","['10', 'years', 'ago,', 'took', 'task', 'teach...",1535


## Attempt to load and train a doc2vec model from gensim
This model is going to be used to encode each of the documents from the ted talks dataset. The below code is from [TutorialsPoint gensim doc2vec](https://www.tutorialspoint.com/gensim/gensim_doc2vec_model.htm). We can directly see that it creates embeddings through the infer_vector method. This method takes in a list of tokens. This works well for us since we already tokenized each of the Ted Talk transcripts.

We can also see that during the training of the Doc2vec model we can specify the size of the encoding vector. This will allow us to fine tune things as needed.

In [8]:
import gensim
import gensim.downloader as api
dataset = api.load("text8")
data = [d for d in dataset]
def tagged_document(list_of_list_of_words):
   for i, list_of_words in enumerate(list_of_list_of_words):
      yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
data_for_training = list(tagged_document(data))
# print(data_for_training[:1])
model = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)
model.build_vocab(data_for_training)
model.train(data_for_training, total_examples=model.corpus_count, epochs=model.epochs)
print(model.infer_vector(['violent', 'means', 'to', 'destroy', 'the','organization']))

[-1.1383144e-01 -1.4812869e-01 -4.0645540e-01 -4.8280985e-04
  4.7971867e-02 -2.0906860e-01 -5.0884746e-02 -3.3710886e-02
 -8.9957835e-03  1.3318349e-03  8.4077947e-02 -5.5916473e-02
 -3.2418200e-01 -1.7155860e-01 -2.4771862e-01  1.3800710e-01
  1.1050384e-01  9.1013320e-02 -1.4316100e-01 -1.7142878e-01
  7.6005869e-02  7.0684820e-02  2.4357627e-03  9.5974043e-02
 -8.2248449e-02 -7.9889551e-02 -2.2094214e-01  6.7696259e-03
 -5.1605701e-02 -9.0819567e-02 -1.3385870e-01  1.9221224e-02
 -2.9755622e-01 -5.2030206e-01 -3.0527997e-01 -2.8008096e-02
 -9.6790560e-02 -2.5922081e-01  1.2792276e-02 -1.8423110e-01]


### Test on a single Ted Talk transcript before running all of them.

In [9]:
model.infer_vector(ted['tokens'][0].split())

array([ 0.00198165, -0.01197452,  0.00065136,  0.0015177 , -0.01175789,
       -0.01139545,  0.00466623, -0.00935497, -0.01104016, -0.00145649,
       -0.00228846,  0.00609594,  0.0045124 ,  0.00504051, -0.00888881,
        0.00427341,  0.00188837, -0.00381613, -0.0026749 , -0.01128546,
        0.00177312, -0.00438745, -0.00528742,  0.00719001,  0.00897924,
        0.00163171,  0.01161056, -0.00096824, -0.00528743, -0.00704321,
       -0.00569449,  0.01102407,  0.01237641, -0.00826293,  0.01026698,
       -0.00564227, -0.00844584, -0.00386677,  0.00744819,  0.00235513],
      dtype=float32)

## Embed all of the transcripts

In [10]:
embeddings = []
for tokens in tqdm(ted['tokens']):
    embedding = model.infer_vector(tokens.split())
    embeddings.append(embedding)
len(embeddings)

100%|██████████| 2461/2461 [00:10<00:00, 236.23it/s]


2461

In [11]:
ted['embeddings'] = embeddings
ted.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,...,tags,title,urlurl,views,transcript,urlurl.1,transcript_cleaned,tokens,transcript_n_entries,embeddings
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,...,"['children', 'creativity', 'culture', 'dance',...",Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,"good morning. you? it great, it? blown away wh...","['good', 'morning.', 'you?', 'it', 'great,', '...",1484,"[0.0019816533, -0.011974525, 0.0006513625, 0.0..."
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,...,"['alternative energy', 'cars', 'climate change...",Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,"thank much, chris. truly great honor opportuni...","['thank', 'much,', 'chris.', 'truly', 'great',...",1012,"[-0.006541744, -0.009786108, 0.00223324, -0.00..."
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,...,"['computers', 'entertainment', 'interface desi...",Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,"(music: the sound silence, simon & garfunkel)h...","['(music:', 'the', 'sound', 'silence,', 'simon...",1673,"[-0.003987377, 0.0017378584, -0.004346572, -0...."
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,...,"['MacArthur grant', 'activism', 'business', 'c...",Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,todayhappyheard sustainable development save u...,"['todayhappyheard', 'sustainable', 'developmen...",1624,"[-0.0068321778, -0.012379591, -0.011552184, 0...."
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,...,"['Africa', 'Asia', 'Google', 'demo', 'economic...",The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,"10 years ago, took task teach global developme...","['10', 'years', 'ago,', 'took', 'task', 'teach...",1535,"[0.004503235, -0.0074265623, -0.004571422, 0.0..."


## Get a list of all possible tags

In [12]:
tags = []
topics = []
for video in tqdm(ted['tags']):
    video = video.replace('[', '').replace(']', '').replace('\'', '').replace('\"', '').split(', ')
    topics += video
    tags.append(video)
ted['tags'] = tags
topics = list(set(topics))
print(topics[0:10])
print(len(topics))

100%|██████████| 2461/2461 [00:00<00:00, 153819.06it/s]

['big bang', 'curiosity', 'sanitation', 'NASA', 'astronomy', 'success', 'fear', 'forensics', 'oil', 'glacier']
415


## Setup a MultiLabelBinarizer
from [Scikit Learn](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MultiLabelBinarizer.html)

In [13]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
mlb.fit([topics])
print(len(mlb.classes_))
mlb.classes_

415


array(['3d printing', 'AI', 'AIDS', 'Addiction', 'Africa', 'Alzheimers',
       'Anthropocene', 'Asia', 'Autism spectrum disorder', 'Bioethics',
       'Blindness', 'Brand', 'Brazil', 'Buddhism', 'CRISPR',
       'Christianity', 'Criminal Justice', 'DNA', 'Debate', 'Egypt',
       'Europe', 'Foreign Policy', 'Gender equality', 'Gender spectrum',
       'God', 'Google', 'Guns', 'HIV', 'Human body', 'Internet', 'Iran',
       'Islam', 'LGBT', 'MacArthur grant', 'Mars', 'Middle East', 'Moon',
       'NASA', 'Natural resources', 'New York', 'Nobel prize', 'PTSD',
       'Planets', 'Senses', 'Slavery', 'South America', 'String theory',
       'Surgery', 'Surveillance', 'Syria', 'TED Books', 'TED Brain Trust',
       'TED Fellows', 'TED Prize', 'TED Residency', 'TED en Español',
       'TED-Ed', 'TEDMED', 'TEDNYC', 'TEDYouth', 'TEDx', 'Transgender',
       'United States', 'Vaccines', 'activism', 'adventure',
       'advertising', 'aging', 'agriculture', 'aircraft', 'algorithm',
       'alte

## Transform the Ted Talk Tags using the MLB

In [14]:
# first a test
print([ted['tags'][0]])
print(mlb.transform([ted['tags'][0]])[0])

[['children', 'creativity', 'culture', 'dance', 'education', 'parenting', 'teaching']]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0]


In [15]:
tag_encoding = []
for tags in tqdm(ted['tags']):
    tag_encoding.append(mlb.transform([tags])[0])
len(tag_encoding)

100%|██████████| 2461/2461 [00:00<00:00, 8683.67it/s]


2461

In [16]:
ted['tag_encoding'] = tag_encoding
ted.head()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,...,title,urlurl,views,transcript,urlurl.1,transcript_cleaned,tokens,transcript_n_entries,embeddings,tag_encoding
0,4553,Sir Ken Robinson makes an entertaining and pro...,1164,TED2006,1140825600,60,Ken Robinson,Ken Robinson: Do schools kill creativity?,1,1151367060,...,Do schools kill creativity?,https://www.ted.com/talks/ken_robinson_says_sc...,47227110,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,"good morning. you? it great, it? blown away wh...","['good', 'morning.', 'you?', 'it', 'great,', '...",1484,"[0.0019816533, -0.011974525, 0.0006513625, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,265,With the same humor and humanity he exuded in ...,977,TED2006,1140825600,43,Al Gore,Al Gore: Averting the climate crisis,1,1151367060,...,Averting the climate crisis,https://www.ted.com/talks/al_gore_on_averting_...,3200520,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,"thank much, chris. truly great honor opportuni...","['thank', 'much,', 'chris.', 'truly', 'great',...",1012,"[-0.006541744, -0.009786108, 0.00223324, -0.00...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,124,New York Times columnist David Pogue takes aim...,1286,TED2006,1140739200,26,David Pogue,David Pogue: Simplicity sells,1,1151367060,...,Simplicity sells,https://www.ted.com/talks/david_pogue_says_sim...,1636292,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,"(music: the sound silence, simon & garfunkel)h...","['(music:', 'the', 'sound', 'silence,', 'simon...",1673,"[-0.003987377, 0.0017378584, -0.004346572, -0....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,200,"In an emotionally charged talk, MacArthur-winn...",1116,TED2006,1140912000,35,Majora Carter,Majora Carter: Greening the ghetto,1,1151367060,...,Greening the ghetto,https://www.ted.com/talks/majora_carter_s_tale...,1697550,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,todayhappyheard sustainable development save u...,"['todayhappyheard', 'sustainable', 'developmen...",1624,"[-0.0068321778, -0.012379591, -0.011552184, 0....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,593,You've never seen data presented like this. Wi...,1190,TED2006,1140566400,48,Hans Rosling,Hans Rosling: The best stats you've ever seen,1,1151440680,...,The best stats you've ever seen,https://www.ted.com/talks/hans_rosling_shows_t...,12005869,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,"10 years ago, took task teach global developme...","['10', 'years', 'ago,', 'took', 'task', 'teach...",1535,"[0.004503235, -0.0074265623, -0.004571422, 0.0...","[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


## Train a simple Model
Note that we do not want to use a tree based model with our encoded targets because that can cause the tree to be very sparse.

The below cell is used to duplicate the embedding for each of the tags. This is done since I could not figure out the multi-label binarizer.

In [35]:
a = []
b = []
for index, row in tqdm(ted.iterrows()):
    for tag in row['tags']:
        a.append(tag)
        b.append(row['embeddings'])
# for tags, embeddings in tqdm(ted[['tags', 'embeddings']]):
#     for tag in tags:
#         a.append(tag)
#         b.append(embeddings)
print(len(a))
print(len(b))

x = np.array(b)
y = np.array(a)
print(x.shape)
print(y.shape)

2461it [00:00, 10615.96it/s]

18401
18401
(18401, 40)
(18401,)


In [36]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
X_train = np.array(X_train)
print(X_train.shape)
X_train[0]

(12328, 40)


array([-2.4450570e-05,  2.8608486e-03, -3.1430996e-03, -6.8091541e-03,
        9.6280901e-03, -9.8941773e-03,  2.5606467e-03,  7.2913887e-03,
        7.2201790e-04,  8.3412575e-03,  7.1819720e-04,  6.6197081e-03,
        3.5931901e-03,  3.5979166e-03,  1.0761023e-03, -6.5448107e-03,
        8.0086272e-03,  5.1005008e-03,  2.6648776e-03, -8.0012549e-03,
        2.6749312e-03, -5.1172450e-04,  6.0612038e-03, -3.9407783e-03,
        6.2749372e-03, -3.9957613e-03, -1.1406722e-02, -7.7248425e-03,
       -8.9687807e-03,  8.0418186e-03, -8.8271927e-03, -8.2340557e-03,
        9.3849273e-03, -5.1670284e-03,  3.4180209e-03,  4.7874958e-03,
        4.6730549e-03, -9.3517657e-03, -1.6970299e-03, -9.8404894e-03],
      dtype=float32)

In [42]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
# clf = MultiOutputClassifier(LogisticRegression()).fit(X_train, y_train)
# clf = OneVsRestClassifier(LinearSVC()).fit(X_train, y_train)
clf = LogisticRegression().fit(X_train, y_train)

In [43]:
y_pred = clf.predict(X_test)
# try:
#     y_decision = clf.decision_function(X_test)
#     print("ROC_AUC: %.2f" % (metrics.roc_auc_score(y_test, y_decision)))
# except Exception:
#     print("No Decision function. Using predict_proba")
#     y_pred_prob = clf.predict_proba(X_test)
#     print("ROC_AUC: %.2f" % (metrics.roc_auc_score(y_test, y_pred_prob[:,1])))
print("F1: %.2f" % (f1_score(y_test, y_pred, average='weighted')))
print("Accuracy: %.2f" % (metrics.accuracy_score(y_test, y_pred)))
print("Precision: %.2f" % (metrics.precision_score(y_test, y_pred, average='weighted', zero_division=0)))
print("Recall: %.2f" % (metrics.recall_score(y_test, y_pred, average='weighted')))

F1: 0.00
Accuracy: 0.04
Precision: 0.00
Recall: 0.04


In [ ]:
X = ted['embeddings']
# y = ted['tag_encoding']
y = mlb.transform(ted['tags'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
X_train

226     [-0.0076125725, 0.008616281, -0.0018003412, -0...
1313    [0.004246713, -0.011347116, -0.0072755693, 0.0...
2428    [-0.006065508, 0.005633655, -0.0007973455, -0....
715     [-0.0035077282, -0.0021022998, 0.01033441, -0....
1279    [0.011278771, -0.0070805876, -0.003374701, -0....
Name: embeddings, dtype: object

In [ ]:
y_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
# X_train = X_train.iloc[0:5]
# y_train = y_train.iloc[0:5]
# X_train

In [ ]:
# y_train

In [ ]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
# clf = MultiOutputClassifier(LogisticRegression()).fit(X_train, y_train)
# clf = OneVsRestClassifier(LinearSVC()).fit(X_train, y_train)
clf = MultiOutputClassifier(LinearSVC()).fit(X_train, y_train)

ValueError: setting an array element with a sequence.

In [ ]:
# clf = RandomForestClassifier(max_depth=2, random_state=0)
# clf.fit(X_train, y_train)

ValueError: setting an array element with a sequence.